# RunnableParallel

Certainly! RunnableParallel is a powerful tool in LangChain that allows you to run multiple independent chains or runnables concurrently, which can significantly speed up your applications if the tasks don't depend on each other's output. Think of it like this: if you have two chefs who can both start cooking their own dishes at the same time, instead of waiting for one to finish before the other begins, you'll get both dishes faster.

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [1]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [3]:
%pip install -qU langchain-openai

In [4]:
%pip install grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.5 MB/s eta 0:00:00


In [5]:
pip show langchain

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


In [ ]:
# %load_ext dotenv
# %dotenv

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [7]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books.
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers.
    Answer only by listing the projects.
    '''
)

In [8]:
chat = ChatOpenAI(model_name = 'gpt-4',
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
string_parser = StrOutputParser()

In [10]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [11]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [12]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup and Requests\n2. Developing a Text-Based Adventure Game using Object-Oriented Programming\n3. Creating a Personal Finance Tracker using Pandas and Matplotlib'}

In [13]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [14]:
%%time
chain_books.invoke({'programming language':'Python'})

CPU times: user 12.8 ms, sys: 0 ns, total: 12.8 ms
Wall time: 2.27 s


'1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin'

In [15]:
%%time
chain_projects.invoke({'programming language':'Python'})

CPU times: user 10.9 ms, sys: 3.94 ms, total: 14.8 ms
Wall time: 3.25 s


'1. Building a Web Scraper using BeautifulSoup.\n2. Developing a simple GUI-based application using Tkinter.\n3. Creating a basic Machine Learning model using Scikit-learn.'

In [16]:
%%time
chain_parallel.invoke({'programming language':'Python'})

CPU times: user 17.2 ms, sys: 3.03 ms, total: 20.2 ms
Wall time: 3.62 s


{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup and Requests\n2. Developing a simple Machine Learning application with Scikit-learn\n3. Creating a GUI application with Tkinter or PyQt'}